# 7 BANDS Reflectance SMOOTHING ANALYSIS

## Import modules

In [5]:
import array
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from termcolor import colored
import time
import xarray as xr

%reload_ext Cython

## Functions

Cython functions, modified `ws2dvopt` and `ws2dvoptp` in order to get vcurve

In [6]:
%%cython

from cpython.array cimport array, clone
from libc.math cimport log, pow, sqrt
cimport numpy as np
import numpy as np

tFloat = np.double
ctypedef np.double_t dtype_t


cpdef lag1corr(np.ndarray[dtype_t] data1, np.ndarray[dtype_t] data2, double nd):
    """Calculates Lag-1 autocorrelation.

    Adapted from https://stackoverflow.com/a/29194624/5997555

    Args:
        data1: fist data series
        data2: second data series
        nd: no-data value (will be exluded from calulation)

    Returns:
        Lag-1 autocorrelation value
    """

    cdef int M, sub
    cdef double sum1, sum2, var_sum1, var_sum2, cross_sum, std1, std2, cross_mean

    M = data1.size

    sum1 = 0.
    sum2 = 0.
    sub = 0
    for i in range(M):
        if data1[i] != nd and data2[i] != nd:
            sum1 += data1[i]
            sum2 += data2[i]
        else:
            sub += 1
    mean1 = sum1 / (M-sub)
    mean2 = sum2 / (M-sub)

    var_sum1 = 0.
    var_sum2 = 0.
    cross_sum = 0.
    for i in range(M):
        if data1[i] != nd and data2[i] != nd:
            var_sum1 += (data1[i] - mean1) ** 2
            var_sum2 += (data2[i] - mean2) ** 2
            cross_sum += (data1[i] * data2[i])

    std1 = (var_sum1 / (M-sub)) ** .5
    std2 = (var_sum2 / (M-sub)) ** .5
    cross_mean = cross_sum / (M-sub)
    return (cross_mean - mean1 * mean2) / (std1 * std2)

cpdef ws2d(np.ndarray[dtype_t] y, double lmda, np.ndarray[dtype_t] w):
    cdef array dbl_array_template = array('d', [])
    cdef int i, i1, i2, m, n
    cdef array z, d, c, e

    n = y.shape[0]
    m = n - 1

    z = clone(dbl_array_template, n, zero=False)
    d = clone(dbl_array_template, n, zero=False)
    c = clone(dbl_array_template, n, zero=False)
    e = clone(dbl_array_template, n, zero=False)

    d.data.as_doubles[0] = w[0] + lmda
    c.data.as_doubles[0] = (-2 * lmda) / d.data.as_doubles[0]
    e.data.as_doubles[0] = lmda /d.data.as_doubles[0]
    z.data.as_doubles[0] = w[0] * y[0]
    d.data.as_doubles[1] = w[1] + 5 * lmda - d.data.as_doubles[0] * (c.data.as_doubles[0] * c.data.as_doubles[0])
    c.data.as_doubles[1] = (-4 * lmda - d.data.as_doubles[0] * c.data.as_doubles[0] * e.data.as_doubles[0]) / d.data.as_doubles[1]
    e.data.as_doubles[1] =  lmda / d.data.as_doubles[1]
    z.data.as_doubles[1] = w[1] * y[1] - c.data.as_doubles[0] * z.data.as_doubles[0]
    for i in range(2, m-1):
        i1 = i - 1
        i2 = i - 2
        d.data.as_doubles[i]= w[i] + 6 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
        c.data.as_doubles[i] = (-4 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1])/ d.data.as_doubles[i]
        e.data.as_doubles[i] =  lmda / d.data.as_doubles[i]
        z.data.as_doubles[i] = w[i] * y[i] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 2
    i2 = m - 3
    d.data.as_doubles[m - 1] = w[m - 1] + 5 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    c.data.as_doubles[m - 1] = (-2 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1]) / d.data.as_doubles[m - 1]
    z.data.as_doubles[m - 1] = w[m - 1] * y[m - 1] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 1
    i2 = m - 2
    d.data.as_doubles[m] = w[m] +  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    z.data.as_doubles[m] = (w[m] * y[m] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]) / d.data.as_doubles[m]
    z.data.as_doubles[m - 1] = z.data.as_doubles[m - 1] / d.data.as_doubles[m - 1] - c.data.as_doubles[m - 1] * z.data.as_doubles[m]
    for i in range(m-2, -1, -1):
        z.data.as_doubles[i] = z.data.as_doubles[i] / d.data.as_doubles[i] - c.data.as_doubles[i] * z.data.as_doubles[i + 1] - e.data.as_doubles[i] * z.data.as_doubles[i + 2]
    return z

cpdef ws2dp(np.ndarray[dtype_t] y, double lmda, np.ndarray[dtype_t] w, double p):
  """Whittaker smoother with asymmetric smoothing and fixed lambda (S).

  Args:
      y: time-series numpy array
      l: smoothing parameter lambda (S)
      w: weights numpy array
      p: "Envelope" value

  Returns:
      Smoothed time-series array z
  """
  cdef array template = array('d', [])
  cdef int m, i, j
  cdef double y_tmp, z_tmp, p1

  m = y.shape[0]
  i = 0
  j = 0
  p1 = 1-p

  template = array('d', [])
  z = clone(template, m, True)
  znew = clone(template, m, True)
  wa = clone(template, m, False)
  ww = clone(template, m, False)

  # Calculate weights

  for i in range(10):
    for j in range(m):
      y_tmp = y[j]
      z_tmp = z.data.as_doubles[j]

      if y_tmp > z_tmp:
        wa.data.as_doubles[j] = p
      else:
        wa.data.as_doubles[j] = p1
      ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

    znew[0:m] = _ws2d(y, lmda, ww)
    z_tmp = 0.0
    j = 0
    for j in range(m):
      z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

    if z_tmp == 0.0:
      break

    z[0:m]= znew[0:m]

  z[0:m] = _ws2d(y, lmda, ww)
  return z


cdef _ws2d(np.ndarray[dtype_t] y, double lmda, array[double] w):
    """Internal whittaker function for use in asymmetric smoothing.
    Args:
      y: time-series numpy array
      lmbda: lambda (s) value
      w: weights numpy array
    Returns:
        smoothed time-series array z
    """

    cdef array dbl_array_template = array('d', [])
    cdef int i, i1, i2, m, n
    cdef array z, d, c, e

    n = y.shape[0]
    m = n - 1

    z = clone(dbl_array_template, n, zero=False)
    d = clone(dbl_array_template, n, zero=False)
    c = clone(dbl_array_template, n, zero=False)
    e = clone(dbl_array_template, n, zero=False)

    d.data.as_doubles[0] = w.data.as_doubles[0] + lmda
    c.data.as_doubles[0] = (-2 * lmda) / d.data.as_doubles[0]
    e.data.as_doubles[0] = lmda /d.data.as_doubles[0]
    z.data.as_doubles[0] = w.data.as_doubles[0] * y[0]
    d.data.as_doubles[1] = w.data.as_doubles[1] + 5 * lmda - d.data.as_doubles[0] * (c.data.as_doubles[0] * c.data.as_doubles[0])
    c.data.as_doubles[1] = (-4 * lmda - d.data.as_doubles[0] * c.data.as_doubles[0] * e.data.as_doubles[0]) / d.data.as_doubles[1]
    e.data.as_doubles[1] =  lmda / d.data.as_doubles[1]
    z.data.as_doubles[1] = w.data.as_doubles[1] * y[1] - c.data.as_doubles[0] * z.data.as_doubles[0]
    for i in range(2, m-1):
        i1 = i - 1
        i2 = i - 2
        d.data.as_doubles[i]= w.data.as_doubles[i] + 6 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
        c.data.as_doubles[i] = (-4 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1])/ d.data.as_doubles[i]
        e.data.as_doubles[i] =  lmda / d.data.as_doubles[i]
        z.data.as_doubles[i] = w.data.as_doubles[i] * y[i] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 2
    i2 = m - 3
    d.data.as_doubles[m - 1] = w.data.as_doubles[m - 1] + 5 *  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    c.data.as_doubles[m - 1] = (-2 *  lmda - d.data.as_doubles[i1] * c.data.as_doubles[i1] * e.data.as_doubles[i1]) / d.data.as_doubles[m - 1]
    z.data.as_doubles[m - 1] = w.data.as_doubles[m - 1] * y[m - 1] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]
    i1 = m - 1
    i2 = m - 2
    d.data.as_doubles[m] = w.data.as_doubles[m] +  lmda - (c.data.as_doubles[i1] * c.data.as_doubles[i1]) * d.data.as_doubles[i1] - (e.data.as_doubles[i2] * e.data.as_doubles[i2]) * d.data.as_doubles[i2]
    z.data.as_doubles[m] = (w.data.as_doubles[m] * y[m] - c.data.as_doubles[i1] * z.data.as_doubles[i1] - e.data.as_doubles[i2] * z.data.as_doubles[i2]) / d.data.as_doubles[m]
    z.data.as_doubles[m - 1] = z.data.as_doubles[m - 1] / d.data.as_doubles[m - 1] - c.data.as_doubles[m - 1] * z.data.as_doubles[m]
    for i in range(m-2, -1, -1):
        z.data.as_doubles[i] = z.data.as_doubles[i] / d.data.as_doubles[i] - c.data.as_doubles[i] * z.data.as_doubles[i + 1] - e.data.as_doubles[i] * z.data.as_doubles[i + 2]
    return z


cpdef ws2doptv(np.ndarray[dtype_t] y, np.ndarray[dtype_t] w, array[double] llas):
    """Whittaker smoother with normal V-curve optimization of lambda (S).
    Args:
        y: time-series numpy array
        w: weights numpy array
        llas: array with lambda values to iterate (S-range)
    Returns:
        Smoothed time-series array z and optimized lambda (S) value lopt
    """
    cdef array template = array('d', [])
    cdef array fits, pens, diff1, lamids, v, z
    cdef int m, m1, m2, nl, nl1, lix, i, k
    cdef double w_tmp, y_tmp, z_tmp, z2, llastep, f1, f2, p1, p2, l, l1, l2, vmin, lopt

    m = y.shape[0]
    m1 = m - 1
    m2 = m - 2
    nl = len(llas)
    nl1 = nl - 1
    i = 0
    k = 0

    template = array('d', [])

    fits = clone(template, nl, True)
    pens = clone(template, nl, True)
    z = clone(template, m, False)
    diff1 = clone(template, m1, True)
    lamids = clone(template, nl1, False)
    v = clone(template, nl1, False)

    # Compute v-curve
    for lix in range(nl):
        l = pow(10,llas.data.as_doubles[lix])
        z[0:m] = ws2d(y, l, w)
        for i in range(m):
            w_tmp = w[i]
            y_tmp = y[i]
            z_tmp = z.data.as_doubles[i]
            fits.data.as_doubles[lix] += pow(w_tmp * (y_tmp - z_tmp),2) 
        fits.data.as_doubles[lix] = log(fits.data.as_doubles[lix])

        for i in range(m1):
            z_tmp = z.data.as_doubles[i]
            z2 = z.data.as_doubles[i+1]
            diff1.data.as_doubles[i] = z2 - z_tmp
        for i in range(m2):
            z_tmp = diff1.data.as_doubles[i]
            z2 = diff1.data.as_doubles[i+1]
            pens.data.as_doubles[lix] += pow(z2 - z_tmp,2)
        pens.data.as_doubles[lix] = log(pens.data.as_doubles[lix])

    # Construct v-curve
    llastep = llas[1] - llas[0]

    for i in range(nl1):
        l1 = llas.data.as_doubles[i]
        l2 = llas.data.as_doubles[i+1]
        f1 = fits.data.as_doubles[i]
        f2 = fits.data.as_doubles[i+1]
        p1 = pens.data.as_doubles[i]
        p2 = pens.data.as_doubles[i+1]
        v.data.as_doubles[i] = sqrt(pow(f2 - f1,2) + pow(p2 - p1,2)) / (log(10) * llastep)
        lamids.data.as_doubles[i] = (l1+l2) / 2

    vmin = v.data.as_doubles[k]
    for i in range(1, nl1):
        if v.data.as_doubles[i] < vmin:
            vmin = v.data.as_doubles[i]
            k = i

    lopt = pow(10, lamids.data.as_doubles[k])

    z[0:m] = ws2d(y, lopt, w)

    return z, lopt, v, lamids



cpdef ws2doptvp(np.ndarray[dtype_t] y, np.ndarray[dtype_t] w, array[double] llas, double p):
    """Whittaker smoother with asymmetric V-curve optimization of lambda (S).
    Args:
        y: time-series numpy array
        w: weights numpy array
        llas: array with lambda values to iterate (S-range)
        p: "Envelope" value
    Returns:
        Smoothed time-series array z and optimized lambda (S) value lopt
    """
    cdef array template = array('d', [])
    cdef array fits, pens, diff1, lamids, v, z
    cdef int m, m1, m2, nl, nl1, lix, i, j, k
    cdef double w_tmp, y_tmp, z_tmp, z2, llastep, fit1, fit2, pen1, pen2, l, l1, l2, vmin, lopt, p1

    m = y.shape[0]
    m1 = m - 1
    m2 = m - 2
    nl = len(llas)
    nl1 = nl - 1
    i = 0
    k = 0
    j = 0
    p1 = 1-p

    template = array('d', [])
    fits = clone(template, nl, True)
    pens = clone(template, nl, True)
    z = clone(template, m, True)
    znew = clone(template, m, True)
    diff1 = clone(template, m1, True)
    lamids = clone(template, nl1, False)
    v = clone(template, nl1, False)
    wa = clone(template, m, False)
    ww = clone(template, m, False)

    # Compute v-curve
    for lix in range(nl):
        l = pow(10,llas.data.as_doubles[lix])

        for i in range(10):
          for j in range(m):
            y_tmp = y[j]
            z_tmp = z.data.as_doubles[j]
            if y_tmp > z_tmp:
              wa.data.as_doubles[j] = p
            else:
              wa.data.as_doubles[j] = p1
            ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

          znew[0:m] = _ws2d(y, l, ww)
          z_tmp = 0.0
          j = 0
          for j in range(m):
            z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

          if z_tmp == 0.0:
            break

          z[0:m]= znew[0:m]

        for i in range(m):
            w_tmp = w[i]
            y_tmp = y[i]
            z_tmp = z.data.as_doubles[i]
            fits.data.as_doubles[lix] += pow(w_tmp * (y_tmp - z_tmp),2)
        fits.data.as_doubles[lix] = log(fits.data.as_doubles[lix])

        for i in range(m1):
            z_tmp = z.data.as_doubles[i]
            z2 = z.data.as_doubles[i+1]
            diff1.data.as_doubles[i] = z2 - z_tmp
        for i in range(m2):
            z_tmp = diff1.data.as_doubles[i]
            z2 = diff1.data.as_doubles[i+1]
            pens.data.as_doubles[lix] += pow(z2 - z_tmp,2)
        pens.data.as_doubles[lix] = log(pens.data.as_doubles[lix])

    # Construct v-curve
    llastep = llas[1] - llas[0]

    for i in range(nl1):
        l1 = llas.data.as_doubles[i]
        l2 = llas.data.as_doubles[i+1]
        fit1 = fits.data.as_doubles[i]
        fit2 = fits.data.as_doubles[i+1]
        pen1 = pens.data.as_doubles[i]
        pen2 = pens.data.as_doubles[i+1]
        v.data.as_doubles[i] = sqrt(pow(fit2 - fit1,2) + pow(pen2 - pen1,2)) / (log(10) * llastep)
        lamids.data.as_doubles[i] = (l1+l2) / 2

    vmin = v.data.as_doubles[k]
    for i in range(1, nl1):
        if v.data.as_doubles[i] < vmin:
            vmin = v.data.as_doubles[i]
            k = i

    lopt = pow(10, lamids.data.as_doubles[k])

    del z
    z = clone(template, m, True)

    for i in range(10):
      for j in range(m):
        y_tmp = y[j]
        z_tmp = z.data.as_doubles[j]

        if y_tmp > z_tmp:
          wa.data.as_doubles[j] = p
        else:
          wa.data.as_doubles[j] = p1
        ww.data.as_doubles[j] = w[j] * wa.data.as_doubles[j]

      znew[0:m] = _ws2d(y, lopt, ww)
      z_tmp = 0.0
      j = 0
      for j in range(m):
        z_tmp += abs(znew.data.as_doubles[j] - z.data.as_doubles[j])

      if z_tmp == 0.0:
        break

      z[0:m]= znew[0:m]

    z[0:m] = _ws2d(y, lopt, ww)
    
    return z, lopt, v, lamids

Extract time series functions

In [7]:
def select_satellite(sat: str, MD: tuple):
    
    (MOD, MYD, MXD) = MD
    
    if (sat == 'MYD'):
        df = MYD
    elif (sat == 'MOD'):
        df = MOD
    else:
        df = MXD
        
    return df


def extract_ts(df: pd.DataFrame, location: str, date_begin: int, date_end: int):
    
    b1 = df['B1'].loc[location].values
    b2 = df['B2'].loc[location].values
    b3 = df['B3'].loc[location].values
    b4 = df['B4'].loc[location].values
    b5 = df['B5'].loc[location].values
    b6 = df['B6'].loc[location].values
    b7 = df['B7'].loc[location].values

    dts = df['Date'].loc[location].values
    
    c_dts = df['Composite_date'].loc[location].values
    
    #Crop to date range
    
    date_range = np.all([dts>=datetime.date(date_begin,1,1), dts<=datetime.date(date_end,12,31)], axis=0)
    b1 = b1[date_range]
    b2 = b2[date_range]
    b3 = b3[date_range]
    b4 = b4[date_range]
    b5 = b5[date_range]
    b6 = b6[date_range]
    b7 = b7[date_range]
    dts = dts[date_range]
    c_dts = c_dts[date_range]
    
    return (b1, b2, b3, b4, b5, b6, b7, dts, c_dts)

Smoothing functions

In [8]:
def ref_smoothing(b1: np.ndarray, 
                  b2: np.ndarray, 
                  b3: np.ndarray, 
                  b4: np.ndarray, 
                  b5: np.ndarray, 
                  b6: np.ndarray, 
                  b7: np.ndarray, 
                  pvalue: float, nopval: bool, lrange: np.ndarray, nd: int):

    # create weights
    w1 = np.array((b1!=nd)*1,dtype='double')
    w2 = np.array((b2!=nd)*1,dtype='double')
    w3 = np.array((b3!=nd)*1,dtype='double')
    w4 = np.array((b4!=nd)*1,dtype='double')
    w5 = np.array((b5!=nd)*1,dtype='double')
    w6 = np.array((b6!=nd)*1,dtype='double')
    w7 = np.array((b7!=nd)*1,dtype='double')

    # apply whittaker filter with V-curve
    if (nopval):
        z1, lopt1, vcurve1, l1 = ws2doptv(b1, w1, array.array('d',lrange))
        z2, lopt2, vcurve2, l2 = ws2doptv(b2, w2, array.array('d',lrange))
        z3, lopt3, vcurve3, l3 = ws2doptv(b3, w3, array.array('d',lrange))
        z4, lopt4, vcurve4, l4 = ws2doptv(b4, w4, array.array('d',lrange))
        z5, lopt5, vcurve5, l5 = ws2doptv(b5, w5, array.array('d',lrange))
        z6, lopt6, vcurve6, l6 = ws2doptv(b6, w6, array.array('d',lrange))
        z7, lopt7, vcurve7, l7 = ws2doptv(b7, w7, array.array('d',lrange))
    else:
        z1, lopt1, vcurve1, l1 = ws2doptvp(b1, w1, array.array('d',lrange), pvalue)
        z2, lopt2, vcurve2, l2 = ws2doptvp(b2, w2, array.array('d',lrange), pvalue)
        z3, lopt3, vcurve3, l3 = ws2doptvp(b3, w3, array.array('d',lrange), pvalue)
        z4, lopt4, vcurve4, l4 = ws2doptvp(b4, w4, array.array('d',lrange), pvalue)
        z5, lopt5, vcurve5, l5 = ws2doptvp(b5, w5, array.array('d',lrange), pvalue)
        z6, lopt6, vcurve6, l6 = ws2doptvp(b6, w6, array.array('d',lrange), pvalue)
        z7, lopt7, vcurve7, l7 = ws2doptvp(b7, w7, array.array('d',lrange), pvalue)
    
    return (np.array(z1), np.array(z2), np.array(z3), np.array(z4), np.array(z5), np.array(z6), np.array(z7),
            lopt1, lopt2, lopt3, lopt4, lopt5, lopt6, lopt7,
            vcurve1, vcurve2, vcurve3, vcurve4, vcurve5, vcurve6, vcurve7,
            l1, l2, l3, l4, l5, l6, l7)

Print and Plot functions

In [9]:
def print_info(location: str, latlon: dict):
    
    print('\033[1m' + 'Selected Point: ', location, '\033[0m')
    print('(Lat, Lon) =', latlon[location] )
    print('\n')    



def plot_all(Bands: list, Bands_smoothed: list, 
             b:list,
             z: list,
             ref_c_dts: np.ndarray,
             nd: float, 
             yauto: bool, ylimits: tuple):
    
    
    plt.figure(figsize=(20,5))
    
    #smoothed values
    col_raw = ['r', 'blueviolet', 'b', 'g', 'orange', 'greenyellow', 'k']
    col_smoothed = ['r', 'blueviolet', 'b', 'g', 'orange', 'greenyellow', 'k']
    label1 = ['Band1', 'Band2', 'Band3', 'Band4', 'Band5', 'Band6', 'Band7']
    label2 = ['Band1 smoothed', 'Band2 smoothed', 'Band3 smoothed', 'Band4 smoothed', 'Band5 smoothed', 'Band6 smoothed', 'Band7 smoothed']
    leg = []
    
    for i,a in enumerate(Bands):
        if a:
            bnan = b[i].copy()
            bnan[bnan == nd] = np.nan
            plt.plot(ref_c_dts, bnan, color = col_raw[i], marker = 'o', alpha = 0.2)
            leg.append(label1[i])
    
    for i,a in enumerate(Bands_smoothed):
        if a:
            znan = z[i].copy()
            znan[znan == nd] = np.nan
            plt.plot(ref_c_dts, znan, color = col_smoothed[i], alpha = 0.6)
            leg.append(label2[i])
    
    if not(yauto):
        plt.ylim(ylimits)
    
    plt.xlabel('Date', fontsize=15)
    plt.ylabel('Reflectance', fontsize=15)
    plt.legend(leg, fontsize=17, loc = 'lower right')
    plt.show()


    
def plot_vcurve(Bands: list, Bands_smoothed: list, 
                l: list,
                vcurve: list,
                lopt: list):
    
    plt.figure(figsize=(20,10))
        
    col_raw = ['r', 'blueviolet', 'b', 'g', 'orange', 'greenyellow', 'k']
    label = ['Band1', 'Band2', 'Band3', 'Band4', 'Band5', 'Band6', 'Band7']
    leg = []
    
    for i,a in enumerate(Bands):
        if a:
            plt.plot(l[i], vcurve[i], color = col_raw[i], alpha = 0.5, marker = 'o')
            leg.append(label[i] + 'lopt: ' + str(round(np.log10(lopt[i]),2)))
            
    for i,a in enumerate(Bands):
        if a:
            plt.axvline(x = np.log10(lopt[i]), ls = '--', color = col_raw[i])

    plt.xlabel('log10(l)', fontsize=15)
    plt.ylabel('V', fontsize=15)
    plt.title('V-curves', fontsize=23)
    plt.legend(leg, fontsize=17, loc = 'lower right')
    plt.show()
            
    
def plot_year(Bands: list, Bands_smoothed: list,
              year: int, month: int, 
              b:list,
              z: list,
              ref_c_dts: np.ndarray,
              nd: float, 
              yauto: bool, ylimits: tuple):
    
    
    plt.figure(figsize=(20,10))

    # Cropping time series to year
    year_index = np.all([ref_c_dts>=datetime.date(year,month,1), ref_c_dts<datetime.date(year+1,month,1)], axis=0)
    ref_c_dts = ref_c_dts[year_index]
    for i in range(7):
        b[i] = b[i][year_index]
        z[i] = z[i][year_index]
            
    
    #smoothed values
    col_raw = ['r', 'blueviolet', 'b', 'g', 'orange', 'greenyellow', 'k']
    col_smoothed = ['r', 'blueviolet', 'b', 'g', 'orange', 'greenyellow', 'k']
    label1 = ['Band1', 'Band2', 'Band3', 'Band4', 'Band5', 'Band6', 'Band7']
    label2 = ['Band1 smoothed', 'Band2 smoothed', 'Band3 smoothed', 'Band4 smoothed', 'Band5 smoothed', 'Band6 smoothed', 'Band7 smoothed']
    leg = []
    
    for i,a in enumerate(Bands):
        if a:
            bnan = b[i].copy()
            bnan[bnan == nd] = np.nan
            plt.plot(ref_c_dts, bnan, color = col_raw[i], marker = 'o', alpha = 0.2)
            leg.append(label1[i])
    
    for i,a in enumerate(Bands_smoothed):
        if a:
            znan = z[i].copy()
            znan[znan == nd] = np.nan
            plt.plot(ref_c_dts, znan, color = col_smoothed[i], alpha = 0.6)
            leg.append(label2[i])
    
    if not(yauto):
        plt.ylim(ylimits)


    plt.xlabel('Date', fontsize=15)
    plt.ylabel('Reflectance', fontsize=15)
    plt.legend(leg, fontsize=15, loc = 'lower right')
    plt.title('Year ' + str(year), fontsize = 20)
    

Main function

In [11]:
def main(Band1: bool, Band1_smoothed:bool, 
         Band2: bool, Band2_smoothed:bool,
         Band3: bool, Band3_smoothed:bool,
         Band4: bool, Band4_smoothed:bool,
         Band5: bool, Band5_smoothed:bool,
         Band6: bool, Band6_smoothed:bool,
         Band7: bool, Band7_smoothed:bool,
         MD: tuple,
         location: str, latlon: dict, satellite: str, 
         pvalue:float, nopval: bool,
         lrange: tuple, step: float, 
         nd: int, 
         date_begin: int, date_end: int, 
         year: int, month: int,
         yauto: bool, ylimits: tuple):
    
    lrange = np.arange(lrange[0],lrange[1], step)
    
    #extract time series
    df = select_satellite(satellite, MD)
    (b1, b2, b3, b4, b5, b6, b7, ref_dts, ref_c_dts) = extract_ts(df, location, date_begin, date_end)
    
    #smoothing
    (z1, z2, z3, z4, z5, z6, z7,
    lopt1, lopt2, lopt3, lopt4, lopt5, lopt6, lopt7,
    vcurve1, vcurve2, vcurve3, vcurve4, vcurve5, vcurve6, vcurve7,
    l1, l2, l3, l4, l5, l6, l7) = ref_smoothing(b1, b2, b3, b4, b5, b6, b7, pvalue, nopval, lrange, nd)
    
    #Prints and plots
    print_info(location, latlon)
    
    Bands = [Band1, Band2, Band3, Band4, Band5, Band6, Band7]
    Bands_smoothed = [Band1_smoothed, Band2_smoothed, Band3_smoothed, Band4_smoothed, Band5_smoothed, Band6_smoothed, Band7_smoothed]
    b = [b1, b2, b3, b4, b5, b6, b7]
    z = [z1, z2, z3, z4, z5, z6, z7]
    lopt = [lopt1, lopt2, lopt3, lopt4, lopt5, lopt6, lopt7]
    vcurve = [vcurve1, vcurve2, vcurve3, vcurve4, vcurve5, vcurve6, vcurve7]
    l = [l1, l2, l3, l4, l5, l6, l7]
    
    plot_all(Bands, Bands_smoothed, b, z, ref_c_dts, nd, yauto, ylimits)
    plot_year(Bands, Bands_smoothed, year, month, b, z, ref_c_dts, nd, yauto, ylimits)
    plot_vcurve(Bands, Bands_smoothed, l, vcurve, lopt)
    